# SVM
## 寻找最大间隔
$${argmax}_{w,b}\{min_n(y \cdot (w^Tx + b)) \cdot \frac1{\left \| x \right \|}\}$$

## 一般的SVM
SMO算法要解如下凸二次规划的对偶问题：
$$min_\alpha \frac 12 \sum_{i=1}^N \sum_{j=1}^N \alpha_i \alpha_j y_i y_j K(x_i, x_j) - \sum_{i=1}^N \alpha_i$$
$$s.t. \sum_{i=1}^N \alpha_i y_i = 0$$
$$0 \leqslant \alpha_i \leqslant C, \space i=1,2...N$$
SMO算法是一种启发式算法，其基本思路是：如果所有变量的解都满足此最优化问题的KKT条件，那么这个最优化问题的解就得到了。因为KKT条件是该最优化问题的充分必要条件。否则，选择两个变量，固定其他变量，针对这两个变量构建一个凸二次规划问题。这个凸二次规划问题关于这两个变量的解应该更接近原始二次规划问题的解，因为这会使得原二次规划问题的目标函数值变得更小。重要的是，这时子问题可以通过解析方法求解，这样就可以大大提高整个算法的计算速度。

子问题有两个变量，一个是违反KKT条件最严重的那个，另一个有约束条件自动确定。如此，SMO算法将原问题不断分解为子问题并对子问题求解，进而达到求解原问题的目的。

注意，子问题两个变量中只有一个是自由变量。假设$\alpha_1,\alpha_2$为两个变量，其他变量固定，那么由等式约束$\sum_{i=1}^N \alpha_i y_i = 0$可知 $\alpha_1 = -y_1 \sum_{i=2}^N \alpha_i y_i$，如果$\alpha_2$确定，那么$\alpha_1$也随之确定。所以子问题中同时更新两个变量。

整个SMO算法包含两个部分：**求解两个变量的二次解析方法和选择变量的启发式方法**。

In [1]:
import numpy as np

In [2]:
def load_dataset(file_name):
    dataset, labels = [], []
    f = open(file_name)
    for line in f.readlines():
        line_list = line.strip().split('\t')
        dataset.append([float(line_list[0]), float(line_list[1])])
        labels.append([float(line_list[2])])
    return dataset, labels

In [3]:
dataset, labels = load_dataset('svm-dataset.txt')

In [4]:
len(dataset), len(labels)

(100, 100)

In [5]:
dataset[0]

[3.542485, 1.977398]

In [9]:
def select_random(i, n_sample):
    """选择一个[0, m)范围内不等于i的随机数"""
    j = i
    while j == i:
        j = int(np.random.uniform(0, n_sample))
    return j

In [7]:
def clip_alpha(alpha, H, L):
    if alpha > H:
        alpha = H
    if alpha < L:
        alpha = L
    return alpha

### SMO函数的伪代码
```
创建一个alpha向量并将其初始化为0向量
当迭代次数小于最大迭代次数时(外循环）
    对数据集中的每个数据向量（内循环）：
        如果该数据向量可以被优化（满足KKT条件）：
            随机选择另外一个数据向量
            同时优化这两个向量
            如果两个向量都不能被优化，退出内循环
    如果所有向量都没有被优化，进入下一次迭代
```

### 相关公式
预测值: $\hat y = \sum_{i=1}^N \alpha_i y_i x \cdot x_i + b$

In [72]:
def smo_simple(dataset, labels, C, tolerant, max_iter):
    """
    Argument:
        datasets: 数据集
        labels: 标签集
        C: 惩罚系数
        tolerant: 容错率
        max_iter: 最大迭代次数
    """
    data_matrix = np.mat(dataset)
    label_matrix = np.mat(labels)
    b = 0
    n_sample, n_feature = data_matrix.shape
    alphas = np.mat(np.zeros((n_sample, 1)))
    iter_ = 0  # 迭代计数器
    while iter_ < max_iter:
        alpha_pairs_changed_flag = False
        for i in range(n_sample):  # 遍历所有样本
            # 预测值
            y_pred_i = float(np.multiply(alphas, label_matrix).T *
                             (data_matrix * data_matrix[i, :].T)) + b
            # 误差
            error_i = y_pred_i - float(label_matrix[i])
            if (((label_matrix[i] * error_i < -tolerant) and (alphas[i] < C)) or
                    ((label_matrix[i] * error_i > tolerant) and (alphas[i] > 0))):  # 最优化条件
                j = select_random(i, n_sample)  # 随机选择第二个alpha
                y_pred_j = float(np.multiply(alphas, label_matrix).T *
                                 (data_matrix * data_matrix[j, :].T)) + b
                error_j = y_pred_j - float(label_matrix[j])
                alpha_i_old = alphas[i].copy()
                alpha_j_old = alphas[j].copy()

                '''截断，保证alpha在0与C之间'''
                if label_matrix[i] != label_matrix[j]:
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H:
                    print("L==H")
                    continue

                '''修改两个alpha'''
                eta = (2.0 * data_matrix[i, :] * data_matrix[j, :].T
                       - data_matrix[i, :] * data_matrix[i, :].T
                       - data_matrix[j, :] * data_matrix[j, :].T)
                if eta >= 0:
                    print("eta>=0")
                    continue
                alphas[j] -= label_matrix[j] * (error_i - error_j) / eta
                alphas[j] = clip_alpha(alphas[j], H, L)
                if abs(alphas[j] - alpha_pairs_changed_flag) < 0.00001:
                    print("j not moving enough")
                    continue
                # update i by the same amount as j
                # the update is in the oppostie direction
                alphas[i] += label_matrix[j] * label_matrix[i] * (alpha_j_old - alphas[j])

                '''修改对应的b'''
                b1 = (b - error_i -
                      label_matrix[i] * (alphas[i] - alpha_i_old) * data_matrix[i, :] * data_matrix[i, :].T
                      - label_matrix[j] * (alphas[j] - alpha_j_old) * data_matrix[i, :] * data_matrix[j, :].T)
                b2 = (b - error_j -
                      label_matrix[i] * (alphas[i] - alpha_i_old) * data_matrix[i, :] * data_matrix[j, :].T
                      - label_matrix[j] * (alphas[j] - alpha_j_old) * data_matrix[j, :] * data_matrix[j, :].T)

                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                alpha_pairs_changed_flag += 1
                print("iter: %d i:%d, pairs changed %d" % (iter_, i, alpha_pairs_changed_flag))
        if alpha_pairs_changed_flag == 0:
            iter_ += 1
        else:
            iter_ = 0
        print("iteration number: %d" % iter_)
    return b, alphas

In [73]:
b, alphas = smo_simple(dataset, labels, 0.6, 0.001, 40)

L==H
iter: 0 i:1, pairs changed 1
L==H
L==H
L==H
L==H
iter: 0 i:10, pairs changed 2
iter: 0 i:13, pairs changed 3
iter: 0 i:14, pairs changed 4
iter: 0 i:17, pairs changed 5
iter: 0 i:18, pairs changed 6
L==H
L==H
iter: 0 i:25, pairs changed 7
iter: 0 i:26, pairs changed 8
iter: 0 i:27, pairs changed 9
iter: 0 i:29, pairs changed 10
iter: 0 i:54, pairs changed 11
iter: 0 i:55, pairs changed 12
iter: 0 i:92, pairs changed 13
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
L==H
iter: 2 i:54, pairs changed 1
iter: 2 i:55, pairs changed 2
iteration number: 0
iter: 0 i:1, pairs changed 1
iter: 0 i:5, pairs changed 2
iter

In [75]:
b

matrix([[-3.8196884]])

In [76]:
alphas[alphas > 0]

matrix([[  4.42709564e-02,   3.37105576e-01,   1.34006551e-04,
           9.95407784e-02,   2.72257709e-01]])

### 支持向量
支持向量对应的$\alpha > 0$，可得对应的支持向量样本数据

In [77]:
for i in range(len(dataset)):
    if alphas[i] > 0:
        print(dataset[i], labels[i])

[4.658191, 3.507396] [-1.0]
[3.457096, -0.082216] [-1.0]
[2.893743, -1.643468] [-1.0]
[5.286862, -2.358286] [1.0]
[6.080573, 0.418886] [1.0]


### 备注
SVM是在是太难了，其SMO算法的改进部分，和非线性SVM部分先略过